In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE47184'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"In silico nano-dissection: defining cell type specificity at transcriptional level in human disease (tubulointerstitium)"
!Series_summary	"To identify genes with cell-lineage-specific expression not accessible by experimental micro-dissection, we developed a genome-scale iterative method, in-silico nano-dissection, which leverages high-throughput functional-genomics data from tissue homogenates using a machine-learning framework."
!Series_summary	"This study applied nano-dissection to chronic kidney disease and identified transcripts specific to podocytes, key cells in the glomerular filter responsible for hereditary proteinuric syndromes and acquired CKD. In-silico prediction accuracy exceeded predictions derived from fluorescence-tagged-murine podocytes, identified genes recently implicated in hereditary glomerular disease and predicted genes significantly correlated with kidney function. The nano-dissection method is broadly applicable to defin

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Determine if gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# Sample Characteristics Dictionary from the output of STEP 1
sample_characteristics = {
    0: ['tissue: Tubulointerstitium from kidney biopsy'], 
    1: [None, 'disease status: Diabetic Nephropathy', 'disease status: Minimal Change Disease', 'disease status: Thin Membrane Disease', 'disease status: Tumor Nephrectomy', 'disease status: Focal and Segmental Glomerulosclerosis', 'disease status: Hypertensive nephropathy', 'disease status: IgA Nephropathy', 'disease status: Membranous Glomerulonephritis']
}

# 2.1 Data Availability
# Assuming dataset key for Chronic_kidney_disease
trait_row = 1

def convert_trait(value):
    try:
        val = value.split(': ')[1].strip().lower()
        if 'nephropathy' in val or 'glomerulonephritis' in val or 'disease' in val:
            return 1
        elif 'tumor nephrectomy' in val:
            return 0
        else:
            return None
    except:
        return None

# No mention of age and gender data, setting them unavailable
age_row = None

def convert_age(value):
    return None  # As there is no age data available

gender_row = None

def convert_gender(value):
    return None  # As there is no gender data available

# 3. Save Metadata 
save_cohort_info('GSE47184', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE47184.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1146387': [1], 'GSM1146388': [1], 'GSM1146389': [1], 'GSM1146390': [1], 'GSM1146391': [1], 'GSM1146392': [1], 'GSM1146393': [1], 'GSM1146394': [1], 'GSM1146395': [1], 'GSM1146396': [1], 'GSM1146397': [1], 'GSM1146398': [None], 'GSM1146399': [None], 'GSM1146400': [None], 'GSM1146401': [1], 'GSM1146402': [1], 'GSM1146403': [1], 'GSM1146404': [1], 'GSM1146405': [1], 'GSM1146406': [1], 'GSM1146407': [1], 'GSM1146408': [1], 'GSM1146409': [1], 'GSM1146410': [1], 'GSM1146411': [1], 'GSM1146412': [1], 'GSM1146413': [1], 'GSM1146414': [1], 'GSM1146415': [1], 'GSM1146416': [1], 'GSM1146417': [1], 'GSM1146418': [1], 'GSM1146419': [1], 'GSM1146420': [1], 'GSM1146421': [1], 'GSM1146422': [1], 'GSM1146423': [1], 'GSM1146424': [1], 'GSM1146425': [1], 'GSM1146426': [1], 'GSM1146427': [0], 'GSM1146428': [0], 'GSM1146429': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['10', '100', '1000', '10000', '10001', '10002', '10003', '10004',
       '10005', '10006', '10007', '10009', '1001', '10010', '10013', '10014',
       '10015', '10016', '10017', '10018'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '9', '10', '12'], 'GeneID': ['1', '2', '9', '10', '12'], 'ORF': ['A1BG', 'A2M', 'NAT1', 'NAT2', 'SERPINA3'], 'Description': ['alpha-1-Bglycoprotein', 'alpha-2-macroglobulin', 'N-acetyltransferase1(arylamineN-acetyltransferase)', 'N-acetyltransferase2(arylamineN-acetyltransferase)', 'serpinpeptidaseinhibitor,cladeA(alpha-1antiproteinase,antitrypsin),member3'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Chronic_kidney_disease/gene_data/GSE47184.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Chronic_kidney_disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE47184', './preprocessed/Chronic_kidney_disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Chronic_kidney_disease/GSE47184.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Chronic_kidney_disease', the least common label is '0.0' with 3 occurrences. This represents 7.50% of the dataset.
The distribution of the feature 'Chronic_kidney_disease' in this dataset is severely biased.

